# Polariton Peak Analysis

Author: Garrek Stemo

Affiliation: Nara Institute of Science and Technology

Date created: August 31, 2020

Date updated: November 20, 2020

This is an interactive notebook for analyzing angle-resolved FTIR cavity-coupled molecular spectra. Interactivity with a dataset is crucials for deciding on a model and visually inspecting the fits. The notebook uses the [lmfit](https://lmfit.github.io/lmfit-py/) package, which is a wrapper for SciPy's optimize method. It allows the user to easily define and modify a model, making it well-adapted to an interactive programming scheme like this one. Lmfit comes with almost all of the fitting functions one might need to perform analysis, but I have included additional functions in a custom `pmath.py` module, including some asymmetric peak models. Functions to pull data from FTIR experiments and transfer matrix simulations are in the `data_io.py` module. These functions assume the files are named a certain way and are in a particular format, so check out that module for more detail or you can write your own. This routine formats the results in a pandas DataFrame, which is exported as a csv. The main text file for storing Rabi splitting and related metadata is an integral part of the I/O and analysis. It is also loaded as a pandas DataFrame: a central part of the data organization scheme for this research.

## Setup

We use the matplotlib widgets framework to generate interactive plots. Make sure you have the appropriate dependencies installed, like nodejs, Jupyter lab extensions, etc.

In [ ]:
import ipywidgets as widgets
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from scipy import signal
from scipy import optimize
import lmfit as lm
from ruamel_yaml import YAML
import data_io
import material_properties as mp
import pmath

yaml = YAML()
sns.set_theme(context='notebook', style='whitegrid', palette='dark')
%matplotlib widget

## Load Data

Assign `data_directory` the path to a directory containing your angle-resolved data. Each .csv data file must contain `deg##.##_` where the `##.##` is an angle (e.g. 5_, 5.0_, 05.00_, etc.). The underscore is necessary, since the program uses this to distinguish the angle information from the rest of the file name.

In [ ]:
# Use sample to reference this data set throughout the analytical flow

data_directory = '/path/to/angle/resolved/data/directory/'
angle_data, absorbance_data = data_io.get_angle_data_from_dir(data_directory)

print('Loading data from:')
print(data_directory, '\n')
print('Angles in range: [', angle_data[0][0], ',', angle_data[-1][0], ']')

## Setting up your data for fitting

Limit the frequency axis bounds using a single (usually the first, or lowest angle) spectrum. Use these initial tools to zoom around your data and truncate (the bounds are applied globally) to isolate the peaks you want to fit. This is a crucial step, since if there are other peaks in your data that are not included in your model, the fit will be poor.

In [ ]:
# Choose which angle spectrum and subrange to examine

angle_index = 0

lower_bound = 500
upper_bound = 8000
spectrum = angle_data[angle_index]
angle, wavenumber_O, transmittance_O = spectrum
mask = (wavenumber_O >= lower_bound) & (wavenumber_O <= upper_bound)

# Plot the selected spectrum or a subrange
fig1, ax = plt.subplots()

ax.plot(wavenumber_O[mask], transmittance_O[mask])

ax.set_title("Spectrum subrange for {} degrees".format(np.round(angle, 2)))
ax.set_xlabel(r'Wavenumber (cm$^{-1}$)')
ax.set_ylabel('Transmittance %')
plt.show()

## Define the Model and Parameters

Now test the fitting functions available in `pmath.py` or lmfit to find the one you would like to use. The lmfit package has lots of built-in functions. You can use these, fitting functions in pmath, or your own. This is the model you should use to fit spectra for all the other angles in this angle-resolved dataset. Perform the initial fit for the first data set and plot the results before fitting all data sets (residuals are also included in the lmfit package, conveniently). You can decide if you need to adjust your initial guesses or the model. You can test how well the model performs for any angle in your data set.

In [ ]:
# Set the model, make initial guesses for the fit parameters, and apply constraints.

frequency = wavenumber_O[mask]
y = transmittance_O[mask]

model = lm.models.LorentzianModel(prefix='l1_') + lm.models.LorentzianModel(prefix='l2_')
result = model.fit(y, x=frequency, l1_amplitude=1, l1_center=1, l1_sigma=1.0,
                               l2_amplitude=1, l2_center=1, l2_sigma=1.0)

l1_center = np.round(result.params['l1_center'].value, 2)
l2_center = np.round(result.params['l2_center'].value, 2)
peak_diff = np.round(np.abs(l1_center - l2_center), 2)

result.plot()
plt.show()

In [ ]:
# Get the numerical results from this fit

print(result.params.pretty_print())
print('Peak difference: {} - {} = {}'.format(l2_center, l1_center, peak_diff))

## Automatically Fit Peaks From Multiple Data Sets

Now that we have a good fit for the first spectrum, go through and fit the rest, using the results from this first spectrum as the initial guess for the next set, and so on.

Afterwards, we should inspect the results to make sure the fitting procedure worked. It's not feasible to inspect *every* spectrum for large data sets (I won't stop you!), but you can sample as many as you like and see what they look like by changing the `examine_spectrum` index and generating a plot for the raw data (blue dots), the best fit (red), and the initial fit (black dashed line). 

### Define a fitting procedure

Once this is set, you don't have to run this cell anymore.

In [ ]:
def fit_sets(data_sets, model_func, guess, lbound, ubound):
    """
    Fits all data according to a given model. Model and guess follow lmfit framework.
    """
    guess = guess
    results_list = []
    xy_data = []
    num_sets = len(data_sets)
    for i in range(num_sets):
        angle = data_sets[i][0]
        mask = (wavenumber_O >= lbound) & (wavenumber_O <= ubound)
        x_data = data_sets[i][1][mask]
        y_data =data_sets[i][2][mask]
        result = model_func.fit(y_data, x=x_data,
                                l1_amplitude=guess['l1_amplitude'],
                                l1_center=guess['l1_center'],
                                l1_sigma=guess['l1_sigma'],
                                l2_amplitude=guess['l2_amplitude'],
                                l2_center=guess['l2_center'],
                                l2_sigma=guess['l2_sigma'])

        results_list.append(result)
        xy_data.append((angle, x_data, y_data))
        guess = result.values
    return results_list, xy_data

### Analyze all angle-resolved data and Visualize the Results

This will run through all of the angles and fit them using the model defined earlier. Here you can select a subset of data to fit if you don't want to apply the fit to every spectrum.

In [ ]:
freq = wavenumber_O[mask]
intensity = transmittance_O[mask]
sets_to_fit = angle_data[:-1]
fit_lbound = 500
fit_ubound = 8000

initial_result = model.fit(y, x=freq, l1_amplitude=1, l1_center=1, l1_sigma=1.0,
                               l2_amplitude=1, l2_center=1, l2_sigma=1.0)

set_results, set_xy = fit_sets(sets_to_fit, model, initial_result.values, fit_lbound, fit_ubound)

In [ ]:
# Change examine_spectrum to see a specific data set.

examine_spectrum = 10
result = set_results[examine_spectrum]
print(result.params.pretty_print())

angle, wavenumber, transmittance = set_xy[examine_spectrum]
# mask = (wavenumber >= lower_bound) & (wavenumber <= upper_bound)

fig3, ax = plt.subplots()

ax.plot(wavenumber, transmittance, 'bo', markersize=3, label='raw data')
ax.plot(wavenumber, result.init_fit, 'k--', label='initial fit')
ax.plot(wavenumber, result.best_fit, 'r-', label='best fit')

ax.set_title("Spectrum for {} degrees".format(np.round(angle, 1)))
plt.legend()
plt.show()

## Dispersion Relation

Now we plot the peak positions from the fitting procedure as a function of cavity angle and fit the polariton eigenenergies to the upper and lower polariton curves.

In [ ]:
fig4, ax = plt.subplots()

theta = []
LP = []
UP = []

for i in np.arange(len(set_results)):
    theta.append(set_xy[i][0])
    LP.append(set_results[i].params['l1_center'].value)
    UP.append(set_results[i].params['l2_center'].value)
    
ax.scatter(theta, LP)
ax.scatter(theta, UP)
ax.set_xlabel(r'$\theta$ (degrees)')
ax.set_ylabel(r'Wavenumber (cm$^{-1}$)')
ax.set_title('Dispersion Relation')

plt.show()

## Find Rabi splitting using least squares fit on dispersion data

In [ ]:
x0 = [1, 1, 1, 1]    # Initial guess
splitting_fit = pmath.splitting_least_squares(x0, theta, LP, UP)
E0, Ev, Rabi, n_eff = splitting_fit.x

print('E0 =', E0)
print('Ev =', Ev)
print('Splitting =', Rabi)
print('n_eff =', n_eff)

In [ ]:
theta_plot = np.linspace(0, 30, 100)
theta_rad = theta_plot * (np.pi / 180)

En = pmath.coupled_energies(theta_rad, E0, Ev, Rabi, n_eff, branch=0)
Ep = pmath.coupled_energies(theta_rad, E0, Ev, Rabi, n_eff, branch=1)
E_ph = pmath.cavity_mode_energy(theta_rad, E0, n_eff)
E_v = np.full(len(theta_plot), Ev)

fig10, ax = plt.subplots()

ax.plot(theta_plot, En, 'r-')
ax.plot(theta_plot, Ep, 'r-')
ax.plot(theta_plot, E_v, color='dimgray', linestyle='dashed')
ax.plot(theta_plot, E_ph, color='dimgray', linestyle='dashed')
ax.scatter(theta, LP, color='b',s=20)
ax.scatter(theta, UP, color='b', s=20)

rabi_text = r'$\Omega_R = %.2f$' % (Rabi)
ax.text(1, 1, rabi_text, bbox=dict(boxstyle='round, pad=0.5', facecolor='white'))
ax.set_xlabel(r'$\theta$ (degrees)')
ax.set_ylabel(r'Wavenumber (cm$^{-1}$)')
ax.set_title('Dispersion with Fitted Polariton Eigenenergies')

plt.show()

## Export Data

Export polariton fitting data to csv or open the Rabi Splitting Data table and append a new entry.

In [ ]:
# Export dispersion data as .csv and store path in Rabi splitting database for future retrieval

export_to = '/export/path/'

raw_data = pd.DataFrame({'theta': theta,
                        'upper polariton': UP,
                        'lower polariton': LP
                        })

fitted_data = pd.DataFrame({'theta': theta_plot,
                            'upper polariton': Ep,
                            'lower polariton': En,
                            'cavity photon': E_ph,
                            'vibration energy': E_v})

raw_file = export_to + 'filename.csv'
fitted_file = export_to + 'filename_fit.csv'

# raw_data.to_csv(raw_file)
# fitted_data.to_csv(fitted_file)

### Congratulations! You made it to the end of your analysis!